In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [ ]:
browser = webdriver.Chrome('/Users/sky/Downloads/chromedriver')
url = 'https://www.jobkorea.co.kr/Top100/?Main_Career_Type=1&Search_Type=1&BizJobtype_Bctgr_Code=10016&BizJobtype_Bctgr_Name=IT·인터넷&BizJobtype_Code=0&BizJobtype_Name=IT·인터넷+전체&Major_Big_Code=0&Major_Big_Name=전체&Major_Code=0&Major_Name=전체&Edu_Level_Code=9&Edu_Level_Name=전체&Edu_Level_Name=전체&MidScroll=0'
browser.get(url)
soup = BeautifulSoup(browser.page_source)

In [ ]:
rank_rookie = []

for tag in soup.select('#devTypeTab_1 > div.rankListWrap > div.rankListArea.devSarterTab > ol > li '):
    company = {}    
    company['rank'] = tag.select('div > span.num')[0].text
    company['company'] = tag.select('div.co > div.coTit > a')[0].text
    company['title'] = tag.select('div.info > div.tit')[0].text
    company['duty'] = tag.select('div.info > div.sTit')[0].text
    company['career'] = tag.select('div.info > div.sDsc > span')[0].text
    company['degree'] = tag.select('div.info > div.sDsc > span')[1].text
    company['work_place'] = tag.select('div.info > div.sDsc > span')[2].text
    company['work_type'] = tag.select('div.info > div.sDsc > span')[3].text

    company['due_date'] = tag.select('div.side > span.day')[0].text
    
    rank_rookie.append(company)
    
rank_rookie

In [ ]:
df = pd.DataFrame(rank_rookie)
df = df.replace('\n', '', regex=True)
df.to_csv('rank_rookie.csv', encoding='utf8')

In [ ]:
# csv 파일 읽기
df = pd.read_csv('/Users/sky/class_python/1.Crawling/rank_rookie_jobkorea.csv', index_col=0, encoding='utf-8')
# 난잡한거 조금 정리
df.drop(['rank'],axis=1, inplace=True)
df = df.replace('스크랩', '', regex=True)
df.to_csv('rank_rookie_jobkorea.csv', encoding='utf8')

In [ ]:
import pymysql
import pandas as pd

In [ ]:
# 데이터 읽기
rookie_j = pd.read_csv('/Users/sky/class_python/1.Crawling/rank_rookie_jobkorea.csv', index_col=0, encoding='utf-8')
rookie_j.head(3)

In [ ]:
# DB연동
conn = pymysql.connect(host='localhost',
                       database='test', # 자기 데이터 베이스에 맞게 접속
                       user='root',
                       passwd='mlb10200', #자기 비밀번호
                       charset='utf8',
                       local_infile=1) #혹시나 안되면 이거 켜서 접속

cursor = conn.cursor()

In [ ]:
# test용 TABLE 생성
sql = '''
    CREATE TABLE rookie_j(
        company varchar(255),
        title varchar(255),
        career varchar(255),
        degree varchar(255),
        work_type varchar(255),
        work_place varchar(255),
        duty varchar(255),
        due_date varchar(255)
    )
      '''
cursor.execute(sql)
#커밋은 필수
conn.commit()

In [ ]:
# DB INSERT

value_list = []
for index, row in rookie_j.iterrows():
    tu = ( row.company, row.title, row.duty, row.career,
           row.degree, row.work_place, row.work_type, row.due_date ) 
    value_list.append(tu)
    
cursor.executemany('''
                  INSERT INTO rookie_j (company, title, duty, career, degree, work_place, work_type, due_date)
                  values ( %s, %s, %s, %s, %s, %s, %s, %s )
                  ''', value_list)

conn.commit()
#conn.close()

In [ ]:
# 테이블 조회

sql = 'SELECT * FROM test.rookie_j'
cursor.execute(sql)
result = cursor.fetchall()
print(result)
#conn.close()